This example has been adapted to use the [Women's Clothing Review Dataset](https://github.com/AFAgarap/ecommerce-reviews-analysis/blob/7062594/Womens%20Clothing%20E-Commerce%20Reviews.csv) from https://github.com/ray-project/ray/blob/b9a4f64/python/ray/train/examples/transformers/transformers_example.py (which was adapted to use Ray Train from https://github.com/huggingface/transformers/blob/75259b4/examples/pytorch/text-classification/run_glue_no_trainer.py)

In [ ]:
%pip install torch transformers pandas datasets accelerate scikit-learn mlflow tensorboard ray[all]

In [ ]:
!ls -al

In [ ]:
# restart kernel to pick up the pip installs above
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [ ]:
import ray

import argparse
import logging
import math
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

from typing import Dict, Any
import random
import torch

import datasets
import ray
import transformers
from accelerate import Accelerator
from datasets import load_dataset, load_metric
from ray.train import Trainer
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from transformers.utils.versions import require_version

In [ ]:
model_name_or_path = "roberta-base"
use_slow_tokenizer = False
per_device_train_batch_size = 32
learning_rate = 5e-5
weight_decay = 0.0
num_train_epochs = 3
max_train_steps = 100
gradient_accumulation_steps = 1
lr_scheduler_type = "linear"
num_warmup_steps = 0
output_dir = None
seed = None
num_workers = 1
use_gpu = False

data_files = {}
data_files["train"] = "Womens Clothing E-Commerce Reviews.csv"
extension = "csv"

In [ ]:
def train_func():

    accelerator = Accelerator()
    print(accelerator.state)

    raw_datasets = load_dataset(extension, data_files=data_files)

    label_list = raw_datasets["train"].unique("sentiment")
    label_list.sort()  # Sort for determinism
    num_labels = len(label_list)

    config = AutoConfig.from_pretrained(
        model_name_or_path, num_labels=num_labels, 
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path, use_fast=not use_slow_tokenizer
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name_or_path,
        config=config,
    )

    sentence1_key, sentence2_key = "review_body", None

    label_to_id = None
    label_to_id = {v: i for i, v in enumerate(label_list)}

    if label_to_id is not None:
        model.config.label2id = label_to_id
        model.config.id2label = {id: label for label, id in config.label2id.items()}

    padding = "max_length" if pad_to_max_length else False

    def preprocess_function(examples):
        texts = (
            (examples[sentence1_key],)
            if sentence2_key is None
            else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(
            *texts, padding=padding, max_length=max_length, truncation=True
        )

        if "sentiment" in examples:
            if label_to_id is not None:
                result["labels"] = [
                    label_to_id[l] for l in examples["sentiment"]
                ]
            else:
                result["labels"] = examples["sentiment"]

        return result

    processed_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        remove_columns=raw_datasets["train"].column_names,
        desc="Running tokenizer on dataset",
    )

    train_dataset = processed_datasets["train"]

    for index in random.sample(range(len(train_dataset)), 3):
        print(f"Sample {index} of the training set: {train_dataset[index]}.")

    if pad_to_max_length:
        data_collator = default_data_collator
    else:
        data_collator = DataCollatorWithPadding(
            tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None)
        )

    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=data_collator,
        batch_size=per_device_train_batch_size,
    )

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

    model, optimizer, train_dataloader = accelerator.prepare(
       model, optimizer, train_dataloader
    )

    num_update_steps_per_epoch = math.ceil(
        len(train_dataloader) / gradient_accumulation_steps
    )
    if max_train_steps is None:
        max_train_steps = num_train_epochs * num_update_steps_per_epoch
    else:
        num_train_epochs = math.ceil(
            max_train_steps / num_update_steps_per_epoch
        )

    lr_scheduler = get_scheduler(
        name=lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_train_steps,
    )

    metric = load_metric("accuracy")

    total_batch_size = (
        per_device_train_batch_size
        * accelerator.num_processes
        * gradient_accumulation_steps
    )

    print("***** Training *****")
    print(f"  Num examples = {len(train_dataset)}")
    print(f"  Num epochs = {num_train_epochs}")
    print(
        f"  Instantaneous batch size per device ="
        f" {per_device_train_batch_size}"
    )
    print(
        f"  Total train batch size (w. parallel, distributed & accumulation) "
        f"= {total_batch_size}"
    )
    print(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
    print(f"  Total optimization steps = {max_train_steps}")

    progress_bar = tqdm(
        range(max_train_steps), disable=not accelerator.is_local_main_process
    )
    completed_steps = 0

    for epoch in range(num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / gradient_accumulation_steps
            accelerator.backward(loss)
            if (
                step % gradient_accumulation_steps == 0
                or step == len(train_dataloader) - 1
            ):
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1

            if completed_steps >= max_train_steps:
                break
        
    if output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
# Connect to a Ray cluster for distributed training.
ray.shutdown()
ray.init(address="ray://localhost:10001",
         runtime_env={"pip": [
                                "torch", 
                                "scikit-learn",
                                "transformers",
                                "pandas",
                                "datasets",
                                "accelerate",
                                "scikit-learn",
                                "mlflow", 
                                "tensorboard"                         
                             ]
                     }
        )

trainer = Trainer("torch", num_workers=num_workers, use_gpu=use_gpu,
                  resources_per_worker={'CPU': 4, 'GPU': 0})
trainer.start()
trainer.run(train_func)